# CNN Ford Hyperparemter Tuning

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Normalization
%matplotlib inline
import seaborn as sns
from keras.models import Sequential 
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, InputLayer, SimpleRNN
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
Ford = pd.read_csv("Ford_Cleaned_Date.csv")
Ford.date = pd.to_datetime(Ford.date)
Ford = Ford.set_index("date")
Ford = Ford.iloc[14:, :] # to remove first 14 days that include NaNs due to some calculations
pd.DataFrame(Ford.isna().sum()).T

,Ford,F-150,Ford Bronco_x,Ford Mustang_x,Ford Stock,Open,High,Low,Close,Volume,...,Dow_MAvg_s_Move,Dow_EMA_Move,Dow_Disparity_Move,Dow_Disparity_s_Move,Dow_RSI_Move,target_1,target_2,target_3,target_4,target_5
0,0,0,0,0,0,6,6,6,6,6,...,0,0,0,0,0,0,0,0,0,0


In [3]:
Ford = Ford.dropna()
Ford = Ford[~(Ford.isin([np.inf, -np.inf]).any(axis=1))] 

In [4]:
Ford = Ford.drop(['target_1', 'target_2', 'target_4', 'target_5'], axis=1)
target_3 = Ford["target_3"]

#splitting into training sets 
column_indices = {name: i for i, name in enumerate(Ford.columns)}

n = len(Ford)
X_train = Ford[0:int(n*0.7)]
X_val = Ford[int(n*0.7):int(n*0.9)]
X_test = Ford[int(n*0.9):]

y_train = target_3[0:int(n*0.7)]
y_val = target_3[int(n*0.7):int(n*0.9)]
y_test = target_3[int(n*0.9):]

In [5]:
Mscaler = MinMaxScaler() # keeps binarys at zero and 1 :)

X_train = pd.DataFrame(Mscaler.fit_transform(X_train), columns = Ford.columns)
X_val = pd.DataFrame(Mscaler.fit_transform(X_val), columns = Ford.columns)

In [7]:
def df_to_X_y2(df, target, window_size=5):
  df_as_np = df.to_numpy() # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df_as_np)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df_as_np[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)

In [6]:
def kbest_creator(k):
    """
    returns list of k best features and the number of efficient principle compents to use with said k features
    """
    # apply SelectKBest class to extract top 40 best features
    bestfeatures = SelectKBest(score_func=f_regression, k=k)
    best_fit = bestfeatures.fit(X_train, y_train)
    best_scores = pd.DataFrame(best_fit.scores_)
    best_columns = pd.DataFrame(Ford.columns)
    
    # concatenate the dataframes for better visualization
    features_score = pd.concat([best_columns, best_scores], axis=1)
    features_score.columns = ['Features', 'Score']  # naming the dataframe columns
    feats = list(features_score.nlargest(k, 'Score')['Features'])

    pca = PCA().fit(X_train[feats])
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res +1
    res
    
    return feats, res

## Model Testing

In [8]:
early_stopping_monitor = EarlyStopping(patience=4)

In [9]:
# Model1

feats, comp = kbest_creator(5)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [10]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model1_1 = Sequential()
model1_1.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_1.add(Flatten())
model1_1.add(Dense(25, activation='relu')) 
model1_1.add(Dense(1, activation='sigmoid'))

model1_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

2022-05-13 14:07:23.123134: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 3s 42ms/step - loss: 0.6941 - binary_accuracy: 0.4969 - val_loss: 0.6877 - val_binary_accuracy: 0.5547
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6845 - binary_accuracy: 0.5703 - val_loss: 0.6875 - val_binary_accuracy: 0.5620
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6812 - binary_accuracy: 0.5642 - val_loss: 0.6862 - val_binary_accuracy: 0.5328
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6782 - binary_accuracy: 0.5845 - val_loss: 0.6850 - val_binary_accuracy: 0.5474
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6746 - binary_accuracy: 0.5866 - val_loss: 0.6849 - val_binary_accuracy: 0.5620
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6729 - binary_accuracy: 0.6029 - val_loss: 0.6874 - val_binary_accuracy: 0.5328
Epoch 7/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6697 - binar

In [11]:
model1_2 = Sequential()
model1_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model1_2.add(Conv1D(filters=15, kernel_size=2, activation='relu'))
# better without pooling layer
model1_2.add(Flatten())
model1_2.add(Dense(30, activation='relu')) 
model1_2.add(Dense(30, activation='relu')) 
model1_2.add(Dense(1, activation='sigmoid'))

model1_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 34ms/step - loss: 0.6933 - binary_accuracy: 0.5010 - val_loss: 0.6900 - val_binary_accuracy: 0.5328
Epoch 2/30
16/16 [==============================] - 0s 15ms/step - loss: 0.6900 - binary_accuracy: 0.5479 - val_loss: 0.6877 - val_binary_accuracy: 0.5693
Epoch 3/30
16/16 [==============================] - 0s 12ms/step - loss: 0.6884 - binary_accuracy: 0.5540 - val_loss: 0.6862 - val_binary_accuracy: 0.5839
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6862 - binary_accuracy: 0.5601 - val_loss: 0.6858 - val_binary_accuracy: 0.5620
Epoch 5/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6844 - binary_accuracy: 0.5519 - val_loss: 0.6867 - val_binary_accuracy: 0.5401
Epoch 6/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6813 - binary_accuracy: 0.5580 - val_loss: 0.6852 - val_binary_accuracy: 0.5474
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss:

In [12]:
# Model2

feats, comp = kbest_creator(10)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [13]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model2_1 = Sequential()
model2_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1.add(Flatten())
model2_1.add(Dense(25, activation='relu')) 
model2_1.add(Dense(1, activation='sigmoid'))

model2_1.compile(optimizer='adagrad', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 24ms/step - loss: 0.7002 - binary_accuracy: 0.4908 - val_loss: 0.6889 - val_binary_accuracy: 0.5036
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6998 - binary_accuracy: 0.4929 - val_loss: 0.6886 - val_binary_accuracy: 0.5036
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6994 - binary_accuracy: 0.4990 - val_loss: 0.6884 - val_binary_accuracy: 0.5182
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6990 - binary_accuracy: 0.4929 - val_loss: 0.6881 - val_binary_accuracy: 0.5182
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6987 - binary_accuracy: 0.4969 - val_loss: 0.6879 - val_binary_accuracy: 0.5255
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6983 - binary_accuracy: 0.5010 - val_loss: 0.6876 - val_binary_accuracy: 0.5328
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6980 - binar

In [14]:
model2_2 = Sequential()
model2_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2.add(Flatten())
model2_2.add(Dense(25, activation='relu')) 
model2_2.add(Dense(1, activation='sigmoid'))

model2_2.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 14ms/step - loss: 0.6991 - binary_accuracy: 0.5132 - val_loss: 0.6913 - val_binary_accuracy: 0.4818
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6892 - binary_accuracy: 0.5560 - val_loss: 0.6906 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6852 - binary_accuracy: 0.5601 - val_loss: 0.6879 - val_binary_accuracy: 0.5109
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6810 - binary_accuracy: 0.6008 - val_loss: 0.6868 - val_binary_accuracy: 0.5109
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6771 - binary_accuracy: 0.5967 - val_loss: 0.6864 - val_binary_accuracy: 0.5182
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6741 - binary_accuracy: 0.6151 - val_loss: 0.6860 - val_binary_accuracy: 0.5474
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6

In [15]:
model2_3 = Sequential()
model2_3.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model2_3.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model2_3.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model2_3.add(Flatten())
model2_3.add(Dense(30, activation='relu')) 
model2_3.add(Dense(1, activation='sigmoid'))

model2_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 19ms/step - loss: 0.6933 - binary_accuracy: 0.4949 - val_loss: 0.6895 - val_binary_accuracy: 0.5547
Epoch 2/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6863 - binary_accuracy: 0.5601 - val_loss: 0.6909 - val_binary_accuracy: 0.5328
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6831 - binary_accuracy: 0.5703 - val_loss: 0.6922 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6803 - binary_accuracy: 0.5886 - val_loss: 0.6928 - val_binary_accuracy: 0.4745
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6789 - binary_accuracy: 0.6069 - val_loss: 0.6930 - val_binary_accuracy: 0.5109


In [16]:
model2_4 = Sequential()
model2_4.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model2_4.add(Conv1D(filters=32, kernel_size=2, activation='relu')) 
model2_4.add(MaxPooling1D(pool_size=2)) 
model2_4.add(Conv1D(filters=32, kernel_size=1, activation='relu')) 
model2_4.add(Flatten())
model2_4.add(Flatten())
model2_4.add(Dense(25, activation='relu')) 
model2_4.add(Dense(1, activation='sigmoid'))

model2_4.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 20ms/step - loss: 0.6928 - binary_accuracy: 0.5010 - val_loss: 0.6917 - val_binary_accuracy: 0.4964
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6880 - binary_accuracy: 0.5336 - val_loss: 0.6908 - val_binary_accuracy: 0.4891
Epoch 3/30
16/16 [==============================] - 0s 17ms/step - loss: 0.6857 - binary_accuracy: 0.5479 - val_loss: 0.6911 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6828 - binary_accuracy: 0.5804 - val_loss: 0.6936 - val_binary_accuracy: 0.4891
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6802 - binary_accuracy: 0.5845 - val_loss: 0.6929 - val_binary_accuracy: 0.4891
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6778 - binary_accuracy: 0.5764 - val_loss: 0.6956 - val_binary_accuracy: 0.4818


In [17]:
# Model3

feats, comp = kbest_creator(25)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [18]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model3_1 = Sequential()
model3_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model3_1.add(Flatten())
model3_1.add(Dense(25, activation='relu')) 
model3_1.add(Dense(1, activation='sigmoid'))

model3_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 27ms/step - loss: 0.6967 - binary_accuracy: 0.4908 - val_loss: 0.6897 - val_binary_accuracy: 0.4964
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6851 - binary_accuracy: 0.5662 - val_loss: 0.6902 - val_binary_accuracy: 0.5328
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6791 - binary_accuracy: 0.5845 - val_loss: 0.6912 - val_binary_accuracy: 0.5036
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6738 - binary_accuracy: 0.6029 - val_loss: 0.6924 - val_binary_accuracy: 0.4964
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6680 - binary_accuracy: 0.6375 - val_loss: 0.6947 - val_binary_accuracy: 0.5036


In [19]:
model3_2 = Sequential()
model3_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model3_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model3_2.add(Flatten())
model3_2.add(Dense(30, activation='relu')) 
model3_2.add(Dense(1, activation='sigmoid'))

model3_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 19ms/step - loss: 0.6972 - binary_accuracy: 0.4664 - val_loss: 0.6909 - val_binary_accuracy: 0.5255
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6901 - binary_accuracy: 0.5092 - val_loss: 0.6917 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6853 - binary_accuracy: 0.5316 - val_loss: 0.6917 - val_binary_accuracy: 0.5255
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6808 - binary_accuracy: 0.5540 - val_loss: 0.6946 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6759 - binary_accuracy: 0.5886 - val_loss: 0.6942 - val_binary_accuracy: 0.5255


In [20]:
# Model4

feats, comp = kbest_creator(50)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [21]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model4_1 = Sequential()
model4_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model4_1.add(Flatten())
model4_1.add(Dense(25, activation='relu')) 
model4_1.add(Dense(1, activation='sigmoid'))

model4_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 14ms/step - loss: 0.6977 - binary_accuracy: 0.5010 - val_loss: 0.6888 - val_binary_accuracy: 0.4964
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6780 - binary_accuracy: 0.5743 - val_loss: 0.6870 - val_binary_accuracy: 0.5036
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6669 - binary_accuracy: 0.6171 - val_loss: 0.6859 - val_binary_accuracy: 0.5036
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6575 - binary_accuracy: 0.6538 - val_loss: 0.6851 - val_binary_accuracy: 0.5182
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6481 - binary_accuracy: 0.6599 - val_loss: 0.6835 - val_binary_accuracy: 0.5620
Epoch 6/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6388 - binary_accuracy: 0.6741 - val_loss: 0.6832 - val_binary_accuracy: 0.5693
Epoch 7/30
16/16 [==============================] - 0s 15ms/step - loss: 0.6290 - bina

In [22]:
model4_2 = Sequential()
model4_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model4_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model4_2.add(Flatten())
model4_2.add(Dense(30, activation='relu')) 
model4_2.add(Dense(1, activation='sigmoid'))

model4_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 16ms/step - loss: 0.6991 - binary_accuracy: 0.5092 - val_loss: 0.6965 - val_binary_accuracy: 0.5328
Epoch 2/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6872 - binary_accuracy: 0.5438 - val_loss: 0.6958 - val_binary_accuracy: 0.4672
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6793 - binary_accuracy: 0.5967 - val_loss: 0.6968 - val_binary_accuracy: 0.4453
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6722 - binary_accuracy: 0.6110 - val_loss: 0.6989 - val_binary_accuracy: 0.4526
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6650 - binary_accuracy: 0.6151 - val_loss: 0.7013 - val_binary_accuracy: 0.4818
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6567 - binary_accuracy: 0.6477 - val_loss: 0.7024 - val_binary_accuracy: 0.4745


In [23]:
# Model5

feats, comp = kbest_creator(100)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [24]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model5_1 = Sequential()
model5_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model5_1.add(Flatten())
model5_1.add(Dense(25, activation='relu')) 
model5_1.add(Dense(1, activation='sigmoid'))

model5_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 2s 14ms/step - loss: 0.7258 - binary_accuracy: 0.4888 - val_loss: 0.7220 - val_binary_accuracy: 0.4599
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6896 - binary_accuracy: 0.5397 - val_loss: 0.7225 - val_binary_accuracy: 0.4672
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6751 - binary_accuracy: 0.5967 - val_loss: 0.7166 - val_binary_accuracy: 0.4307
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6600 - binary_accuracy: 0.6130 - val_loss: 0.7245 - val_binary_accuracy: 0.4453
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6475 - binary_accuracy: 0.6477 - val_loss: 0.7204 - val_binary_accuracy: 0.4234
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6343 - binary_accuracy: 0.6843 - val_loss: 0.7195 - val_binary_accuracy: 0.4526
Epoch 7/30
16/16 [==============================] - 0s 20ms/step - loss: 0.6225 - bina

In [25]:
model5_2 = Sequential()
model5_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model5_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model5_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model5_2.add(Flatten())
model5_2.add(Dense(30, activation='relu')) 
model5_2.add(Dense(1, activation='sigmoid'))

model5_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 28ms/step - loss: 0.7017 - binary_accuracy: 0.5275 - val_loss: 0.7063 - val_binary_accuracy: 0.4088
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6845 - binary_accuracy: 0.5662 - val_loss: 0.7029 - val_binary_accuracy: 0.4380
Epoch 3/30
16/16 [==============================] - 0s 12ms/step - loss: 0.6735 - binary_accuracy: 0.6293 - val_loss: 0.7034 - val_binary_accuracy: 0.4672
Epoch 4/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6633 - binary_accuracy: 0.6395 - val_loss: 0.7054 - val_binary_accuracy: 0.4818
Epoch 5/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6517 - binary_accuracy: 0.6599 - val_loss: 0.7079 - val_binary_accuracy: 0.4964
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6407 - binary_accuracy: 0.7006 - val_loss: 0.7098 - val_binary_accuracy: 0.4964


In [26]:
model5_3 = Sequential()
model5_3.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model5_3.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model5_3.add(MaxPooling1D(pool_size=2)) 
model5_3.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
# better without pooling layer
model5_3.add(Flatten())
model5_3.add(Dense(30, activation='relu')) 
model5_3.add(Dense(1, activation='sigmoid'))

model5_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_3.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 27ms/step - loss: 0.6913 - binary_accuracy: 0.5234 - val_loss: 0.6881 - val_binary_accuracy: 0.5328
Epoch 2/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6826 - binary_accuracy: 0.5886 - val_loss: 0.6890 - val_binary_accuracy: 0.5547
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6762 - binary_accuracy: 0.6232 - val_loss: 0.6886 - val_binary_accuracy: 0.5693
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6685 - binary_accuracy: 0.6599 - val_loss: 0.6885 - val_binary_accuracy: 0.5839
Epoch 5/30
16/16 [==============================] - 0s 17ms/step - loss: 0.6596 - binary_accuracy: 0.6477 - val_loss: 0.6909 - val_binary_accuracy: 0.5766


In [27]:
#Model6 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 3 periods
X_train_1, train_3w = df_to_X_y2(X_train_1,y_train,3)
X_val_1, val_3w = df_to_X_y2(X_val_1, y_val, 3)
X_test_1, test_3w = df_to_X_y2(X_test_1,y_test, 3) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [28]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model6_1 = Sequential()
model6_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model6_1.add(Flatten())
model6_1.add(Dense(25, activation='relu')) 
model6_1.add(Dense(1, activation='sigmoid'))

model6_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model6_1.fit(X_train_1, train_3w,epochs=30,  validation_data=(X_val_1, val_3w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 14ms/step - loss: 0.7146 - binary_accuracy: 0.4929 - val_loss: 0.6964 - val_binary_accuracy: 0.5612
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6935 - binary_accuracy: 0.5213 - val_loss: 0.6956 - val_binary_accuracy: 0.5540
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6855 - binary_accuracy: 0.5416 - val_loss: 0.6953 - val_binary_accuracy: 0.5180
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6793 - binary_accuracy: 0.5659 - val_loss: 0.6949 - val_binary_accuracy: 0.5324
Epoch 5/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6737 - binary_accuracy: 0.5903 - val_loss: 0.6941 - val_binary_accuracy: 0.5683
Epoch 6/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6689 - binary_accuracy: 0.6065 - val_loss: 0.6938 - val_binary_accuracy: 0.5612
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6635 - binar

In [30]:
model6_2 = Sequential()
model6_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model6_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model6_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model6_2.add(Flatten())
model6_2.add(Dense(30, activation='relu')) 
model6_2.add(Dense(1, activation='sigmoid'))

model6_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model6_2.fit(X_train_1, train_3w,epochs=30,  validation_data=(X_val_1, val_3w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 22ms/step - loss: 0.7033 - binary_accuracy: 0.4828 - val_loss: 0.6933 - val_binary_accuracy: 0.5540
Epoch 2/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6919 - binary_accuracy: 0.5436 - val_loss: 0.6912 - val_binary_accuracy: 0.5683
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6846 - binary_accuracy: 0.5740 - val_loss: 0.6898 - val_binary_accuracy: 0.5612
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6786 - binary_accuracy: 0.5943 - val_loss: 0.6889 - val_binary_accuracy: 0.5468
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6730 - binary_accuracy: 0.6166 - val_loss: 0.6884 - val_binary_accuracy: 0.5612
Epoch 6/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6665 - binary_accuracy: 0.6450 - val_loss: 0.6879 - val_binary_accuracy: 0.5683
Epoch 7/30
16/16 [==============================] - 0s 12ms/step - loss: 0

In [31]:
#Model7 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 7 periods
X_train_1, train_7w = df_to_X_y2(X_train_1,y_train,7)
X_val_1, val_7w = df_to_X_y2(X_val_1, y_val, 7)
X_test_1, test_7w = df_to_X_y2(X_test_1,y_test, 7) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [32]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model7_1 = Sequential()
model7_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model7_1.add(Flatten())
model7_1.add(Dense(25, activation='relu')) 
model7_1.add(Dense(1, activation='sigmoid'))

model7_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model7_1.fit(X_train_1, train_7w,epochs=30,  validation_data=(X_val_1, val_7w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 2s 17ms/step - loss: 0.6966 - binary_accuracy: 0.5072 - val_loss: 0.6913 - val_binary_accuracy: 0.4963
Epoch 2/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6784 - binary_accuracy: 0.5726 - val_loss: 0.6897 - val_binary_accuracy: 0.4963
Epoch 3/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6661 - binary_accuracy: 0.6135 - val_loss: 0.6886 - val_binary_accuracy: 0.5556
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6568 - binary_accuracy: 0.6135 - val_loss: 0.6870 - val_binary_accuracy: 0.5333
Epoch 5/30
16/16 [==============================] - 0s 15ms/step - loss: 0.6474 - binary_accuracy: 0.6483 - val_loss: 0.6878 - val_binary_accuracy: 0.5333
Epoch 6/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6391 - binary_accuracy: 0.6728 - val_loss: 0.6865 - val_binary_accuracy: 0.5259
Epoch 7/30
16/16 [==============================] - 0s 23ms/step - loss: 0.6272 - 

In [33]:
model7_2 = Sequential()
model7_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model7_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model7_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model7_2.add(Flatten())
model7_2.add(Dense(30, activation='relu')) 
model7_2.add(Dense(1, activation='sigmoid'))

model7_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model7_2.fit(X_train_1, train_7w,epochs=30,  validation_data=(X_val_1, val_7w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 25ms/step - loss: 0.7001 - binary_accuracy: 0.5072 - val_loss: 0.6919 - val_binary_accuracy: 0.5333
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6816 - binary_accuracy: 0.5971 - val_loss: 0.6923 - val_binary_accuracy: 0.5333
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6712 - binary_accuracy: 0.5971 - val_loss: 0.6916 - val_binary_accuracy: 0.5185
Epoch 4/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6592 - binary_accuracy: 0.6074 - val_loss: 0.6867 - val_binary_accuracy: 0.5259
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6502 - binary_accuracy: 0.6728 - val_loss: 0.6860 - val_binary_accuracy: 0.5852
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6345 - binary_accuracy: 0.7117 - val_loss: 0.6828 - val_binary_accuracy: 0.5407
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.

In [34]:
#Model8 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 14 periods
X_train_1, train_14w = df_to_X_y2(X_train_1,y_train,14)
X_val_1, val_14w = df_to_X_y2(X_val_1, y_val, 14)
X_test_1, test_14w = df_to_X_y2(X_test_1,y_test, 14) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [35]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model8_1 = Sequential()
model8_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model8_1.add(Flatten())
model8_1.add(Dense(25, activation='relu')) 
model8_1.add(Dense(1, activation='sigmoid'))

model8_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model8_1.fit(X_train_1, train_14w,epochs=30,  validation_data=(X_val_1, val_14w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 2s 36ms/step - loss: 0.7046 - binary_accuracy: 0.4917 - val_loss: 0.6915 - val_binary_accuracy: 0.5625
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6626 - binary_accuracy: 0.6266 - val_loss: 0.6888 - val_binary_accuracy: 0.5703
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6449 - binary_accuracy: 0.6805 - val_loss: 0.6989 - val_binary_accuracy: 0.5234
Epoch 4/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6241 - binary_accuracy: 0.7033 - val_loss: 0.7010 - val_binary_accuracy: 0.5469
Epoch 5/30
16/16 [==============================] - 0s 18ms/step - loss: 0.6075 - binary_accuracy: 0.7261 - val_loss: 0.7031 - val_binary_accuracy: 0.5156
Epoch 6/30
16/16 [==============================] - 0s 11ms/step - loss: 0.5891 - binary_accuracy: 0.7510 - val_loss: 0.7069 - val_binary_accuracy: 0.5547


In [36]:
model8_2 = Sequential()
model8_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model8_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model8_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model8_2.add(Flatten())
model8_2.add(Dense(30, activation='relu')) 
model8_2.add(Dense(1, activation='sigmoid'))

model8_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model8_2.fit(X_train_1, train_14w,epochs=30,  validation_data=(X_val_1, val_14w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 30ms/step - loss: 0.6993 - binary_accuracy: 0.4813 - val_loss: 0.6975 - val_binary_accuracy: 0.4766
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6774 - binary_accuracy: 0.5685 - val_loss: 0.7028 - val_binary_accuracy: 0.5000
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6635 - binary_accuracy: 0.6473 - val_loss: 0.6999 - val_binary_accuracy: 0.4844
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6464 - binary_accuracy: 0.7012 - val_loss: 0.7055 - val_binary_accuracy: 0.4375
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6262 - binary_accuracy: 0.7427 - val_loss: 0.7075 - val_binary_accuracy: 0.4766


In [40]:
#Model9 Early RNN
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 14 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train,5)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val, 5)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test, 5) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [43]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model_9 = Sequential()
model_9.add(SimpleRNN(8,activation="relu", return_sequences=False, input_shape=(n_steps,n_features)))
model_9.add(Dense(30, activation='relu')) 
model_9.add(Dense(1, activation='sigmoid'))

model_9.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_9.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 21ms/step - loss: 0.6935 - binary_accuracy: 0.4929 - val_loss: 0.6820 - val_binary_accuracy: 0.5839
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6859 - binary_accuracy: 0.5336 - val_loss: 0.6812 - val_binary_accuracy: 0.5766
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6838 - binary_accuracy: 0.5621 - val_loss: 0.6810 - val_binary_accuracy: 0.5839
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6826 - binary_accuracy: 0.5560 - val_loss: 0.6816 - val_binary_accuracy: 0.5839
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6814 - binary_accuracy: 0.5866 - val_loss: 0.6818 - val_binary_accuracy: 0.5620
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6798 - binary_accuracy: 0.5804 - val_loss: 0.6810 - val_binary_accuracy: 0.6058
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6783 - binar